In [1]:
import scipy as sp
import numpy as np
import pandas as pd
import timeit
import re
import json
import pickle
import fastparquet
import os
os.chdir('/mnt/t48/bighomes-active/sfeng/patentdiffusion/')
seed = 3
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.iolib.summary2 as summary2

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [4]:
r = {}
r["JTH Extension"] = pd.read_pickle("DataStore/2018-07-P3/JTHReg0727/targ_cite_sim_reg_0727.pkl")
r["JTH Cite Sim"] = pd.read_pickle("DataStore/2018-07-P3/JTHReg0727/tp_cite_loc_sim_0727.pkl")
dms = ["ldavecs", "docvecs"]
rs_keys = ["naics_name", "primclass"]
for f in rs_keys:
    for dm in dms:
        r[dm,f] = fastparquet.ParquetFile("DataStore/2018-07-P2/Reg0726/reg_{0}_sim_tr_0726.parq".format(f))\
        .to_pandas(["tp", "tp_naics_name", "tp_primclass", "tp_gyear", "sim_ldavecs", "sim_docvecs",\
            "year_group", "inv_msa_match"])

In [5]:
r["JTH Extension"].columns
y_dict = {"JTH Extension": ["perc_match_10"],
         "JTH Cite Sim": ["sim_ldavecs_tp_cite_10", "sim_docvecs_tp_cite_10"],
         ("ldavecs", "naics_name"): ["sim_ldavecs"],
         ("ldavecs", "primclass"): ["sim_ldavecs"],
         ("docvecs", "naics_name"): ["sim_docvecs"],
         ("docvecs", "primclass"): ["sim_docvecs"],}

repl = pickle.load( open("DataStore/2018-07-P3/reg_names_dict.pkl", "rb"))

ms = pd.DataFrame()
for k,v in r.items():
    for y in y_dict[k]:
        a1 = v[y].describe()
        a2 = v.loc[v["inv_msa_match"] == True, y].describe()
        a3 = v.loc[v["inv_msa_match"] == False, y].describe()
        ms2 = pd.concat([a1,a2,a3], axis=1).T.reset_index()
        ms2["Sample"] = str(k)
        ms2[""] = ["All", "$MSA\, Match = T$", "$MSA\, Match = F$"]
        ms = ms.append(ms2, ignore_index = True)
ms = ms.replace(repl)
ms = np.round(ms,3)
ms["count"] = ms["count"].astype(int)
ms = ms[["Sample", "", "index", 'count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']]
ms.columns = ["Sample", "Subset", "$y$, KS", '$N$', 'Mean', 'Std Dev', 'Min', '25%', '50%', '75%', 'Max']

ms2 = ms.copy()
ms2.columns = ["\thead{"+c+"}" for c in ms2.columns]

In [6]:
display(ms)
print(ms2.to_latex(index=False,escape=False))

,Sample,Subset,"$y$, KS",$N$,Mean,Std Dev,Min,25%,50%,75%,Max
0,JTH Extension,All,"$pct\,cites\,in\,MSA_{T}$",507870,0.082,0.217,0.000,0.000,0.000,0.000,1.000
1,JTH Extension,"$MSA\, Match = T$","$pct\,cites\,in\,MSA_{T}$",282079,0.112,0.250,0.000,0.000,0.000,0.042,1.000
2,JTH Extension,"$MSA\, Match = F$","$pct\,cites\,in\,MSA_{T}$",225791,0.045,0.159,0.000,0.000,0.000,0.000,1.000
3,JTH Cite Sim,All,"$\overline{sim_{LDA}(T,j)}$",283080,0.510,0.201,0.002,0.369,0.509,0.654,1.000
4,JTH Cite Sim,"$MSA\, Match = T$","$\overline{sim_{LDA}(T,j)}$",57897,0.550,0.222,0.002,0.392,0.555,0.721,1.000
5,JTH Cite Sim,"$MSA\, Match = F$","$\overline{sim_{LDA}(T,j)}$",225183,0.500,0.194,0.002,0.365,0.499,0.638,1.000
6,JTH Cite Sim,All,"$\overline{sim_{DV}(T,j)}$",283080,0.289,0.124,-0.552,0.211,0.285,0.362,0.936
7,JTH Cite Sim,"$MSA\, Match = T$","$\overline{sim_{DV}(T,j)}$",57897,0.323,0.151,-0.552,0.224,0.314,0.409,0.936
8,JTH Cite Sim,"$MSA\, Match = F$","$\overline{sim_{DV}(T,j)}$",225183,0.280,0.114,-0.518,0.209,0.279,0.351,0.890
9,NAICS,All,"$sim_{LDA}(i,j)$",1483214,0.242,0.222,0.001,0.061,0.174,0.368,1.000


\begin{tabular}{lllrrrrrrrr}
\toprule
\thead{Sample} &     \thead{Subset} &              \thead{$y$, KS} &  \thead{$N$} &  \thead{Mean} &  \thead{Std Dev} &  \thead{Min} &  \thead{25%} &  \thead{50%} &  \thead{75%} &  \thead{Max} \\
\midrule
 JTH Extension &                All &    $pct\,cites\,in\,MSA_{T}$ &       507870 &         0.082 &            0.217 &        0.000 &        0.000 &        0.000 &        0.000 &        1.000 \\
 JTH Extension &  $MSA\, Match = T$ &    $pct\,cites\,in\,MSA_{T}$ &       282079 &         0.112 &            0.250 &        0.000 &        0.000 &        0.000 &        0.042 &        1.000 \\
 JTH Extension &  $MSA\, Match = F$ &    $pct\,cites\,in\,MSA_{T}$ &       225791 &         0.045 &            0.159 &        0.000 &        0.000 &        0.000 &        0.000 &        1.000 \\
  JTH Cite Sim &                All &  $\overline{sim_{LDA}(T,j)}$ &       283080 &         0.510 &            0.201 &        0.002 &        0.369 &        0.509 &        0.